In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt


class HeatDiffusion(object):
    def __init__(self, noise_steps=500, init_u=1000, alpha=2., delta=1e-4, N_terms=10, device="cuda"):
        self.device = device
        self.noise_steps = noise_steps
        
        self.init_u = init_u
        self.alpha = alpha
        self.delta = delta
        self.N_terms = N_terms
        
        self.N = int(2. / delta)
        
        self.vals = np.linspace(0, 2, self.N)
        self.X, self.Y = np.meshgrid(self.vals, self.vals)
        
    def forward_diffusion(self, x0, t):
        heat = np.array([[self.compute_u(x, y, t, x0) for x in self.vals] for y in self.vals])
        threshold = 0.1 * np.max(heat)
        
        contour = plt.contour(self.X, self.Y, heat, levels=[threshold])
        paths = contour.collections[0].get_paths()
        
        boundaries = []
        for path in paths:
            v = path.vertices
            boundaries.append(v)
            
        return boundaries
        
    
    def compute_A(self, n, m, x0):
        integral = 0
        for i in range(0, self.N):
            for j in range(0, self.N):
                x = i * self.delta
                y = j * self.delta
                integral += self.u0(x, y, x0) * np.sin(n * np.pi * x / 2.) * np.sin(m * np.pi * y / 2.) * self.delta**2
        return integral  
    
    def compute_u(self, x, y, t, x0):
        """
            u(x,y,t) = sum_n sum_m A_nm * sin(n*\pi*x/L_x) * sin(m*\pi*y/L_y) * \
                exp(-\alpha * ((n*\pi/L_x)^2 + (m*\pi/L_y)^2) * t)
        """
        u = 0
        for n in range(1, self.N_terms+1):
            for m in range(1, self.N_terms+1):
                A_nm = self.compute_A(n, m, x0)
                lambda_val = (n * np.pi / 2.)**2 + (m * np.pi / 2.)**2
                u += A_nm * np.sin(n * np.pi * x / 2.) * np.sin(m * np.pi * y / 2.) * np.exp(-self.alpha * lambda_val * t)
        return u
    
    def u0(self, x, y, x0):
        """
            u0(x,y) = u0 * direc(x-x0, y-y0)
        """
        print(float(x0[...,0]))
        if (float(x0[...,0]) <= x < float(x0[...,0])+self.delta) and (float(x0[...,1]) <= y < float(x0[...,1])+self.delta):
            return self.init_u / (self.delta**2)    # normalize
        else:
            raise "wrong x0"
        

    def sample_timesteps(self, n):
        return torch.randint(low=1, high=self.noise_steps, size=(n,)).long()


In [2]:
import torch
heat = HeatDiffusion()
x0 = torch.tensor([[[0.5, 0.5]]], dtype=torch.float, device='cuda')
heat.forward_diffusion(x0, 0)

0.5


TypeError: exceptions must derive from BaseException

In [6]:
import numpy as np
print(np.array([1.]) < 1)

[False]
